## Inspect and processing data manually

In [ ]:
%%sh
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz /tmp

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/tmp/amazon_reviews_us_Camera_v1_00.tsv.gz', sep='\t', 
                   compression='gzip', error_bad_lines=False, dtype='str')
data.dropna(inplace=True)

In [ ]:
print(data.shape)
print(data.columns)

In [ ]:
data.head()

In [ ]:
data = data[:100000]

In [ ]:
data = data[['star_rating', 'review_body']]

In [ ]:
data.star_rating.unique()

In [ ]:
data['label'] = data.star_rating.map({
    '1': '__label__negative__',
    '2': '__label__negative__',
    '3': '__label__neutral__',
    '4': '__label__positive__',
    '5': '__label__positive__'})

In [ ]:
data = data.drop(['star_rating'], axis=1)

In [ ]:
data = data[['label', 'review_body']]

In [ ]:
data.head()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
%%time
data['review_body'] = data['review_body'].apply(nltk.word_tokenize)

In [ ]:
%%time
data['review_body'] = data.apply(lambda row: " ".join(row['review_body']).lower(), axis=1)

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

training, validation = train_test_split(data, test_size=0.05)

In [ ]:
print(training.shape)
print(validation.shape)

In [ ]:
np.savetxt('/tmp/training.txt', training.values, fmt='%s')
np.savetxt('/tmp/validation.txt', validation.values, fmt='%s')

In [ ]:
!head -5 /tmp/training.txt

## Training

In [ ]:
import boto3
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()
bucket = session.default_bucket()

In [ ]:
# Run this cell if you want to use the data you processed manually

prefix = 'amazon-reviews'

s3_train_path = session.upload_data(path='/tmp/training.txt', bucket=bucket, key_prefix=prefix+'/input/train')
s3_val_path = session.upload_data(path='/tmp/validation.txt', bucket=bucket, key_prefix=prefix+'/input/validation')
s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

In [ ]:
# Run this cell if you want to use the data processed by SageMaker Processing

prefix = 'amazon-reviews'

s3_train_path = 
s3_val_path = 
s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

In [ ]:
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('blazingtext', region)
print(container)

In [ ]:
role = sagemaker.get_execution_role()

bt = sagemaker.estimator.Estimator(container,
                                   role, 
                                   instance_count=1, 
                                   instance_type='ml.c5.2xlarge',
                                   output_path=s3_output)

In [ ]:
bt.set_hyperparameters(mode='supervised')

In [ ]:
train_data = sagemaker.TrainingInput(s3_train_path, 
                      distribution='FullyReplicated', 
                      content_type='text/plain',
                      s3_data_type='S3Prefix')

validation_data = sagemaker.TrainingInput(s3_val_path,
                      distribution='FullyReplicated', 
                      content_type='text/plain', 
                      s3_data_type='S3Prefix')

s3_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
bt.fit(inputs=s3_channels)

In [ ]:
bt_predictor = bt.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
import json
import pprint

sentences = ['This is a bad camera it doesnt work at all , i want a refund  .', 
             'The camera works , the pictures are decent quality, nothing special to say about it .',
             'Very happy to have bought this , exactly what I needed']

payload = {"instances" : sentences, "configuration": {"k": 3}}

bt_predictor.serializer = sagemaker.serializers.JSONSerializer()
response = bt_predictor.predict(payload)

print(response)

In [ ]:
bt_predictor.delete_endpoint()